In [52]:
# setup
import hockey_scraper
import requests
import pandas as pd
from py2neo import Graph, Node, Relationship, NodeMatcher


In [54]:
# setup neo4j
driver = Graph(user="neo4j", password="password")
driver.schema.create_uniqueness_constraint("Game", "id")
driver.schema.create_uniqueness_constraint("Play", "id")
#driver.schema.create_uniqueness_constraint("Player", "id")
matcher = NodeMatcher(driver)

In [55]:
# seutp mongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['nhl']
collection = db['pbp']

In [ ]:
# get the 2018/19 gameids
URL = "http://www.nicetimeonice.com/api/seasons/20182019/games"
resp = requests.get(URL)
data = resp.json()

# list of games
games = pd.DataFrame(data)
gids = list(games.gameID)

In [ ]:
# get the json document and throw it into mongodb
for gid in gids:
    GURL = "http://statsapi.web.nhl.com/api/v1/game/{}/feed/live".format(gid)
    resp = requests.get(GURL)
    game = resp.json()
    collection.insert_one(game)
    print("finished game id {}".format(str(gid)))
    


## Parse the game the and the plays

In [57]:
# for each game (document) parse into neo4j
i = 1
for game in collection.find():
    
    #### game, team, and players for the game
    #### https://py2neo.org/v4/database.html?highlight=merge#py2neo.database.Graph.merge
    #### ^^ primary label and key for merge
    
    
    #### live data (plays and players in the play)
    
    ## get the plays
    plays = game['liveData']['plays']['allPlays']

    ## create the game node
    gid = game['gamePk']
    g = Node("Game", id=gid)
    driver.create(g)
    
    # build the plays
    for index, play_data in enumerate(plays):
        play = play_data['result']
        play.update(play_data['about'])
        if 'goals' in play.keys():
          del play['goals']    
        if 'strength' in play.keys():
          del play['strength']    
        play.update(play_data['about']['goals'])
        play.update(play_data['coordinates'])
        play['id'] = "{}_{}".format(gid, index)
        play['gid'] = gid
        play['pid'] = index
        p = Node.cast(play)  ## cast from a dict
        p.add_label("Play") ## add the label I want
        driver.create(p) ## create the node conditionally with trail
        if index == 0:
          r = Relationship(g, "HAD_PLAY", p)
          driver.create(r)
        if index >= 1:
          p0 = matcher.match("Play", pid = index - 1, gid =  gid).first()
          r = Relationship(p0, "NEXT_PLAY", p)
          driver.create(r)
        # parse the players
        if "players" in play_data.keys():
            players = play_data['players']
            for player in players:
                pnode = player['player']
                del player['player']
                player.update(pnode)
                del pnode
                # build the node and the relationship
                player_node = Node.cast(player)
                player_node.add_label("Player")
                driver.create(player_node)
                r_play = Relationship(p, player['playerType'], player_node)
                driver.create(r_play)
            
        
    # status
    print("finished {}".format(str(gid)))
    i = i + 1
    if i == 5:
        break

finished 2018030324
finished 2018030314
finished 2018030323
finished 2018030313


---

In [60]:
game = collection.find_one()

In [61]:
game.keys()

dict_keys(['_id', 'copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [62]:
game['gameData'].keys()

dict_keys(['game', 'datetime', 'status', 'teams', 'players', 'venue'])

In [ ]:
driver.